In [ ]:
import logging
import os
import sys
from pathlib import Path
from typing import Dict

import IPython
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

notebook_path = Path(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"])
project_dir = notebook_path.parent.parent
sys.path.append(str(project_dir))
import src.utils.custom_log as custom_log
from src.utils.Csv import Csv
from src.utils.iso18571 import rating_iso_18571
from src.utils.set_rcparams import set_rcparams

# logging
custom_log.set_level(custom_log.LEVELS.INFO)
LOG: logging.Logger = custom_log.init_logger(__name__)
LOG.info("Log initialized")

# working dir
os.chdir(project_dir)
LOG.info("Working directory: %s", os.getcwd())

# set plot style
set_rcparams()

In [ ]:
B_PATH: Path = Path("/mnt") / "q" / "AWARE2ALL" / "from_esi"
VERSION: str = "2024-06-27_ESI_result_files_v5"
F_NAME: str = "extracted"
PATHS: Dict[str, Path] = {f"VPS {x.parent.stem.split('_')[1]}": x for x in (B_PATH / VERSION).rglob("*.csv.zip")}
PATHS["Dyna HIII 50"] = B_PATH.parent / "original_model_dyna" / F_NAME
LOG.info("Paths: %s", PATHS)

DB: Dict[str, pd.DataFrame] = {}
OUT_DIR = B_PATH / VERSION / "plots"
OUT_DIR.mkdir(exist_ok=True)
LOG.info("Output directory: %s", OUT_DIR)

PATHS

In [ ]:
def read_data():
    db = {}
    for case in PATHS.keys():
        db[case] = Csv(csv_path=PATHS[case], compress=True).read()
        for xx in ("H3", "VI"):
            db[case].rename(columns={v: v.replace(xx, "OC") for v in db[case].columns}, inplace=True)
        for xx in ("05", "50", "95"):
            db[case].rename(columns={v: v.replace(xx, "XX") for v in db[case].columns if "BELT" not in v}, inplace=True)
    return db


DB = read_data()
DB

In [ ]:
def plot():
    ref_case = "Dyna HIII 50"
    iso_min, iso_max = 20, 120
    units = {
        "AC": "Acceleration [g]",
        "DS": "Displacement [mm]",
        "FO": "Force [kN]",
        "IM": "Mass [g]",
        "OM": "Mass [g]",
        "PR": "Pressure [MPa]",
        "TM": "Mass [g]",
        "TP": "Temperature [K]",
        "VE": "Velocity [m/s]",
        "VO": "Volume [l]",
    }
    # get shared channels
    shared_channels = None
    for case in DB.keys():
        if shared_channels is None:
            shared_channels = set(DB[case].columns)
        else:
            shared_channels &= set(DB[case].columns)

    for channel in tqdm(shared_channels, desc="Plotting"):
        fig, ax = plt.subplots()
        for case in DB.keys():
            if case != ref_case:
                iso = rating_iso_18571(
                    signal_ref=DB[ref_case].loc[iso_min:iso_max, channel].values,
                    signal_comp=DB[case].loc[iso_min:iso_max, channel].values,
                )
                label = f"{case} -- ISO18571 to {ref_case}: {iso['ISO 18571 Rating']:.3f}"
            else:
                label = case
            ax.plot(DB[case].index, DB[case][channel], label=label)
        ax.axvline(iso_min, color="black", linestyle="--", alpha=0.5)
        ax.axvline(iso_max, color="black", linestyle="--", alpha=0.5)
        ax.set_title(channel)
        ax.set_xlabel("Time [ms]")
        ax.set_ylabel(units[channel[-4:-2]])
        ax.set_xlim(0, 140)
        ax.set_xticks(range(0, 141, 10))
        ax.legend()
        ax.grid()
        fig.savefig(OUT_DIR / f"{channel}.png")
        plt.close(fig)


plot()